# Import Libraries

In [1]:
import fatbox
import numpy as np 
import networkx as nx
import pickle
import cv2
from PIL import Image
from tqdm import tqdm

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import ListedColormap
from scipy.spatial import distance_matrix
from skimage import feature, morphology, filters
from sklearn.preprocessing import normalize

from sys import stdout

from fatbox.preprocessing import *
from fatbox.metrics import *
from fatbox.edits import *
from fatbox.plots import *

# Processing

## Workflow
1. Open image data as 
2. normalization
3. smoothing, add gaussian blur
4. binarization, threshold
5. skeletonizing
6. removing smaal regions
7. connecting the components
8. make empty nx.Graph, defines point, add the 'pos' and 'components'
9. add edge
10. (iterative) split triple juction & Labeling components, removing 
12. Compute edge length
13. calculate strike
14. comp_to_fault 
15. convert to json and store to somtehin else
16. add inline number to ['pos][x] and make point[1] and point[0] as y and z
17. get node colors, and get node attributes
18. extraxt x,y,z coordinate and store them to an np.array
19. plot in 3d projection (beware of the nodes color, prefer in one color)

## Make the fuunction

In [ ]:
def processing_full():

    # 1. open the data
    """can use iterative function on a list
    containt files and store them to an object"""

    

    # return json file sek
    return none